Solr notes

Solr config:
```
/etc/default/solr.in.sh

SOLR_HOME="/var/solr/data"
```

Create a core:
```
sudo su - solr
/opt/solr/bin/solr create -c isb_rel
```

The set autocreate fields off, with:
```
solr config -c isb_rel -p 8983 \
  -action set-user-property \
  -property update.autoCreateFields \
  -value false
```


In [5]:
import json
import requests

SOLR_API = "http://localhost:8983/api/collections/isb_rel/"
MEDIA_JSON = "application/json"

def pj(o):
    print(json.dumps(o, indent=2))

def listFields():
    headers = {
        "Accept":MEDIA_JSON
    }
    _schema = requests.get(f"{SOLR_API}schema", headers=headers).json()
    return _schema.get("schema",{}).get("fields")

def listFieldTypes():
    headers = {
        "Accept":MEDIA_JSON
    }
    _schema = requests.get(f"{SOLR_API}schema", headers=headers).json()
    return _schema.get("schema",{}).get("fieldTypes")

def createField(fname, ftype="string", stored=True, indexed=True, default=None):
    headers = {
        "Content-Type":MEDIA_JSON
    }
    data = {
    "add-field": {
        "name": fname,
        "type": ftype,
        "stored": stored,
        "indexed": indexed,
    }}
    if not default is None:
        data["add-field"]["default"] = default
    data = json.dumps(data).encode("utf-8")
    res = requests.post(f"{SOLR_API}schema", headers=headers, data=data)
    pj(res.json())

pj(listFields())

[
  {
    "name": "_nest_path_",
    "type": "_nest_path_"
  },
  {
    "name": "_root_",
    "type": "string",
    "docValues": false,
    "indexed": true,
    "stored": false
  },
  {
    "name": "_text_",
    "type": "text_general",
    "multiValued": true,
    "indexed": true,
    "stored": false
  },
  {
    "name": "_version_",
    "type": "plong",
    "indexed": false,
    "stored": false
  },
  {
    "name": "id",
    "type": "string",
    "multiValued": false,
    "indexed": true,
    "required": true,
    "stored": true
  }
]


```
datetime : pdate
URI : string
```

In [6]:
createField("name", "string", True, True, "")
createField("source", "string", True, True)
createField("s", "string", True, True)
createField("p", "string", True, True)
createField("o", "string", True, True)
createField("tstamp", "pdate", True, True)

{
  "responseHeader": {
    "status": 0,
    "QTime": 984
  }
}
{
  "responseHeader": {
    "status": 0,
    "QTime": 1158
  }
}
{
  "responseHeader": {
    "status": 0,
    "QTime": 464
  }
}
{
  "responseHeader": {
    "status": 0,
    "QTime": 902
  }
}
{
  "responseHeader": {
    "status": 0,
    "QTime": 546
  }
}
{
  "responseHeader": {
    "status": 0,
    "QTime": 205
  }
}


In [4]:
def deleteRecords(q="*:*"):
    headers = {
        "Content-Type":MEDIA_JSON
    }
    url = f"http://localhost:8983/solr/isb_rel/update"
    doc = {"delete":{"query":q}}
    doc = json.dumps(doc).encode("utf-8")
    params={"commit":"true"}
    res = requests.post(url, headers=headers, data=doc, params=params)
    print(f"status: {res.status_code}")
    print(res.text)

deleteRecords()

status: 200
{
  "responseHeader":{
    "rf":1,
    "status":0,
    "QTime":31}}

